In [1]:
import shapely.wkt
import os
from osgeo import ogr

GFT_REFRESH_TOKEN = os.environ['GFT_REFRESH_TOKEN']

from osgeo import gdal

ogr.UseExceptions()

# gdal.SetConfigOption('CPL_DEBUG', 'ON')
gdal.SetConfigOption('CPL_DEBUG', 'OFF')

In [2]:
def convert2ft(input_path, output_path, append=False, fix_geometry=False, simplify_geometry=False, start_index=0):
    filename = input_path

    src_ds = ogr.Open(input_path)
    src_lyr = src_ds.GetLayerByIndex(0)
    f = src_lyr.GetFeature(0)

    # create feature table
    dst_ds = ogr.GetDriverByName('GFT').Open('GFT:' + GFT_REFRESH_TOKEN, True)
    
    if append:
        dst_lyr = dst_ds.GetLayerByName(output_path)
    else:
        dst_lyr = dst_ds.CreateLayer(output_path)

        # create fields using OGR
        [dst_lyr.CreateField(f.GetFieldDefnRef(i)) for i in range(f.GetFieldCount())]

    index = 0
    batch_size = 15
    index_batch = 0
    for feat in src_lyr:
        if index < start_index:
            index = index + 1
            continue
        
        try:
            geom = shapely.wkt.loads(feat.GetGeometryRef().ExportToWkt())
        except Exception as e: 
            print('Error({0}), skipping geometry.'.format(e))
            continue

        if fix_geometry and not geom.is_valid:
            geom = geom.buffer(0.0)
        
        if simplify_geometry:
            geom = geom.simplify(0.004)

        f = ogr.Feature(dst_lyr.GetLayerDefn())
        
        # set field values
        for i in range(feat.GetFieldCount()):
            fd = feat.GetFieldDefnRef(i)
            f.SetField(fd.GetName(), feat.GetField(fd.GetName()))
            
            #print(fd.GetName())
            #print(feat.GetField(fd.GetName()))
        
        # set geometry    
        f.SetGeometry(ogr.CreateGeometryFromWkt(geom.to_wkt()))
        
        if index_batch == 0:
            dst_lyr.StartTransaction()
            
        index_batch = index_batch + 1
        
        # create feature
        feature = dst_lyr.CreateFeature(f)
        
        f.Destroy() 
        
        index = index + 1

        if index_batch > batch_size:
            dst_lyr.CommitTransaction()
            index_batch = 0
        
    src_ds.Destroy()

    dst_ds.Destroy()

In [ ]:
input_file = '/home/gennadiy_donchyts/src/GlobalHAND30/all.shp'
output_table = 'max4_2'
append = False
fix_geometry = True
simplify_geometry = False

# gdal.SetConfigOption('CPL_DEBUG', 'OFF')
    
# convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry) 
convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry, 10617) # append

In [ ]:
input_file = '/home/gennadiy_donchyts/src/GlobalHAND30/all.shp'
output_table = 'max4_2'
append = False
fix_geometry = True
simplify_geometry = False

# gdal.SetConfigOption('CPL_DEBUG', 'OFF')
    
# convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry) 
convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry, 10617) # append

In [6]:
ds = ogr.Open(input_file)
l = ds.GetLayerByIndex(0)
f = l.GetFeature(0)


In [7]:
print(l.GetFeatureCount())

19328


In [ ]:
driver = ogr.GetDriverByName('GFT')
dataSource = driver.Open('GFT:' + GFT_REFRESH_TOKEN, True)
print(dataSource)


layerCount = dataSource.GetLayerCount()

print('Layer count: {0}'.format(layerCount))

for i in range(layerCount):
    layer = dataSource.GetLayer(i)
    print('{0}: {1}'.format(i, layer.GetName()))
    
    